# Install Java

In [ ]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y
!java -version

In [ ]:
# Install HBase

In [ ]:
!wget https://downloads.apache.org/hbase/2.5.10/hbase-2.5.10-bin.tar.gz
!tar xvf hbase-2.5.10-bin.tar.gz

In [ ]:
!sudo mv hbase-2.5.10 /opt/hbase

In [2]:
# Update HBase environment configuration
!sed -i 's|# export JAVA_HOME=.*|export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64|' /opt/hbase/conf/hbase-env.sh

In [ ]:
# Verify JAVA_HOME and set if necessary\n",
import os

# Check if JAVA_HOME is set, if not set it to the correct path\n",
java_home = '/usr/lib/jvm/java-11-openjdk-amd64'
if not os.path.exists(java_home):
    raise FileNotFoundError(f'Java path not found: {java_home}')

# Set JAVA_HOME and update PATH\n",
os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = java_home + '/bin:' + os.environ['PATH']

# Verify the java version to confirm setup\n",
!java -version

In [ ]:
# Start HBase
import os
os.chdir('/opt/hbase')
!bin/start-hbase.sh

In [7]:
!jps

29456 HMaster
30352 Jps


In [8]:
# Create HBase Table
hbase_commands = '''
create 'my_table', 'cf1'
list
exit
'''

# Pass the commands to the HBase shell
with open('create_table.txt', 'w') as file:
    file.write(hbase_commands)

!cat create_table.txt | /opt/hbase/bin/hbase shell

HBase Shell
Use "help" to get list of supported commands.
Use "exit" to quit this interactive shell.
For Reference, please visit: http://hbase.apache.org/2.0/book.html#shell
Version 2.5.10, ra3af60980c61fb4be31e0dcd89880f304d01098a, Thu Jul 18 22:45:17 PDT 2024
Took 0.0031 seconds                                                             
hbase:001:0> 
hbase:002:0> create 'my_table', 'cf1'
Created table my_table
Took 0.9232 seconds                                                             
=> Hbase::Table - my_table
hbase:003:0> list
TABLE                                                                           
my_table                                                                        
1 row(s)
Took 0.0209 seconds                                                             
=> ["my_table"]
hbase:004:0> exit
